In [0]:
import requests
import json
from pyspark.sql.types import *
response = requests.get("https://ghoapi.azureedge.net/api/MDG_0000000020?$filter=SpatialDimType%20eq%20%27Country%27%20and%20TimeDim%20ge%202000%20and%20TimeDim%20le%202016")
tbIncidence = response.json()['value']
dfTb = spark.createDataFrame(tbIncidence, schema = StructType([StructField("SpatialDim", StringType(), True),
                                                               StructField("TimeDim", StringType(), True),
                                                               StructField("NumericValue", StringType(), True)]))
# The API only returns things as strings, thus we need to cast these to floats
dfTb = dfTb.withColumn("TimeDim", dfTb["TimeDim"].cast('int')) \
           .withColumn("NumericValue", dfTb["NumericValue"].cast('float')) \
           .withColumnRenamed("NumericValue","TbPer100000") \
           .withColumnRenamed("SpatialDim","Country") \
           .withColumnRenamed("TimeDim","Year")

display(dfTb)

--------------------------------------------------------------------------- 
 JSONDecodeError Traceback (most recent call last)
 <command-4323050764611078> in <module> 
 3 from pyspark . sql . types import * 
 4 response = requests . get ( "https://ghoapi.azureedge.net/api/MDG_0000000020?$filter=SpatialDimType%20eq%20%27Country%27%20and%20TimeDim%20ge%202000%20and%20TimeDim%20le%202016" ) 
 ----> 5 tbIncidence = response . json ( ) [ 'value' ] 
 6 dfTb = spark.createDataFrame(tbIncidence, schema = StructType([StructField("SpatialDim", StringType(), True),
 7 StructField ( "TimeDim" , StringType ( ) , True ) , 

 /databricks/python/lib/python3.8/site-packages/requests/models.py in json (self, **kwargs) 
 898 # used. 
 899 pass 
 --> 900 return complexjson . loads ( self . text , ** kwargs ) 
 901 
 902 @ property

 /usr/lib/python3.8/json/__init__.py in loads (s, cls, object_hook, parse_float, parse_int, parse_constant, object_pairs_hook, **kw) 
 355 parse_int is None and parse_float is None and 
 356 parse_constant is None and object_pairs_hook is None and not kw):
 --> 357 return _default_decoder . decode ( s ) 
 358 if cls is None : 
 359 cls = JSONDecoder

 /usr/lib/python3.8/json/decoder.py in decode (self, s, _w) 
 335 
 336 """
 --> 337 obj , end = self . raw_decode ( s , idx = _w ( s , 0 ) . end ( ) ) 
 338 end = _w ( s , end ) . end ( ) 
 339 if end != len ( s ) : 

 /usr/lib/python3.8/json/decoder.py in raw_decode (self, s, idx) 
 353 obj , end = self . scan_once ( s , idx ) 
 354 except StopIteration as err : 
 --> 355 raise JSONDecodeError ( "Expecting value" , s , err . value ) from None 
 356 return obj , end

 JSONDecodeError : Expecting value: line 1 column 1 (char 0)

In [0]:
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)


def acked(err, msg):
    """ 
        Error callback is used for generic issues for producer errors. 
        
        Parameters:
            err (err): Error flag.
            msg (str): Error message that was part of the callback.
    """
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))

#DO NOT DELETE THIS
from time import sleep
import uuid
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException
import json
from confluent_kafka.admin import AdminClient, NewTopic


#KAFKA variables, Move to the OS variables or configuration
# This will work in local Jupiter Notebook, but in a databrick, hiding config.py is tougher. 
confluentClusterName = "stage3talent"
confluentBootstrapServers = "pkc-ldvmy.centralus.azure.confluent.cloud:9092"
confluentTopicName = "who-tb-test"
schemaRegistryUrl = "https://psrc-gq7pv.westus2.azure.confluent.cloud"
confluentApiKey = "YHMHG7E54LJA55XZ"
confluentSecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"
confluentRegistryApiKey = "YHMHG7E54LJA55XZ"
confluentRegistrySecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"

admin_client = AdminClient({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(uuid.uuid1()),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest',
    'error_cb': error_cb,
})

In [0]:
# topic_list = []

# topic_list.append(NewTopic(confluentTopicName, 1, 3))
# admin_client.create_topics(topic_list)
# futures = admin_client.create_topics(topic_list)


# try:
#     record_metadata = []
#     for k, future in futures.items():
#         # f = i.get(timeout=10)
#         print(f"type(k): {type(k)}")
#         print(f"type(v): {type(future)}")
#         print(future.result())

# except KafkaError:
#     # Decide what to do if produce request failed...
#     print(traceback.format_exc())
#     result = 'Fail'
# finally:
#     print("finally")

type(k): <class 'str'>
type(v): <class 'concurrent.futures._base.Future'>
finally

--------------------------------------------------------------------------- 
 KafkaException Traceback (most recent call last)
 <command-4194586293159138> in <module> 
 12 print ( f"type(k): {type(k)}" ) 
 13 print ( f"type(v): {type(future)}" ) 
 ---> 14 print ( future . result ( ) ) 
 15 
 16 except KafkaError : 

 /usr/lib/python3.8/concurrent/futures/_base.py in result (self, timeout) 
 442 raise CancelledError ( ) 
 443 elif self . _state == FINISHED : 
 --> 444 return self . __get_result ( ) 
 445 else : 
 446 raise TimeoutError ( ) 

 /usr/lib/python3.8/concurrent/futures/_base.py in __get_result (self) 
 387 if self . _exception : 
 388 try : 
 --> 389 raise self . _exception
 390 finally : 
 391 # Break a reference cycle with the exception in self._exception 

 KafkaException : KafkaError{code=TOPIC_ALREADY_EXISTS,val=36,str="Topic 'who-tb-test' already exists."}

In [0]:
p = Producer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(uuid.uuid1()),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest',
    'error_cb': error_cb,
})

DF_Pandas = dfTb.toPandas()

for i in range(len(DF_Pandas)):
    p.produce(confluentTopicName,DF_Pandas.iloc[i].to_json())
    p.flush()
    sleep(5)